In [86]:
import pandas as pd
import numpy as np

from cryptodatapy.extract.datarequest import DataRequest
from cryptodatapy.extract.getdata import GetData
from cryptodatapy.transform.od import OutlierDetection
from cryptodatapy.transform.impute import Impute
from cryptodatapy.transform.filter import Filter
from cryptodatapy.transform.clean import CleanData, stitch_dataframes
from cryptodatapy.transform.impute import Impute

In [94]:
data_req = DataRequest(source='tiingo')

In [95]:
GetData(data_req).get_meta(method='get_top_mkt_cap_info')

AttributeError: 'Tiingo' object has no attribute 'get_top_mkt_cap_info'

In [89]:
fx_df

,,close
date,ticker,
2019-01-31,EURUSD,1.147579
2019-02-28,EURUSD,1.138693
2019-03-31,EURUSD,1.122355
2019-04-30,EURUSD,1.120699
2019-05-31,EURUSD,1.11433
...,...,...
2024-04-30,EURUSD,1.066605
2024-05-31,EURUSD,1.084825
2024-06-30,EURUSD,1.073515


In [2]:
# get all Binance perp futures tickers
data_req = DataRequest(source='ccxt')
perp_tickers = GetData(data_req).get_meta(method='get_markets_info', exch='binanceusdm', as_list=True)

In [3]:
len(perp_tickers)

314

In [4]:
# get Binance spot tickers
data_req = DataRequest(source='ccxt')
spot_tickers = GetData(data_req).get_meta(method='get_markets_info', exch='binance', as_list=True)

In [5]:
# find intersecting tickers
binance_tickers = [ticker for ticker in perp_tickers if ticker in spot_tickers]

In [6]:
# number of tickers
len(binance_tickers)

284

In [7]:
## # get cryptocompare tickers
data_req = DataRequest(source='cryptocompare')
cc_tickers = GetData(data_req).get_meta(method='get_assets_info', as_list=True)

In [8]:
# keep only USDT ticker
bin_tickers = []
for ticker in binance_tickers:
    if '/' in ticker and ticker.split('/')[1] == 'USDT':
        bin_tickers.append(ticker.split('/')[0])

In [9]:
# usdt tickers
usdt_tickers = [ticker.split('/')[0] for ticker in binance_tickers if '/'in ticker and ticker.split('/')[1] == 'USDT']

In [10]:
# intersecting tickers
tickers = [ticker for ticker in usdt_tickers if ticker in cc_tickers]

In [11]:
len(tickers)

257

### Binance Perp Futures

In [12]:
# pull daily OHLC and funding rates for perp futures on Binance USDM exchange
data_req = DataRequest(source='ccxt',
                       tickers=tickers, 
                       fields=['open', 'high', 'low', 'close', 'volume', 'funding_rate'], 
                       mkt_type='perpetual_future', 
                       freq='d')

In [13]:
# df1 = GetData(data_req).get_series()

In [14]:
# df1.to_csv('binance_perp_futures.csv')
df1 = pd.read_csv('../../../../factorlab/notebooks/binance_perp_futures.csv', index_col=['date', 'ticker'], parse_dates=['date'])

In [15]:
df1.head()

,,open,high,low,close,volume,funding_rate
date,ticker,,,,,,
2019-09-08,BTC,10000.00,10412.65,10000.00,10391.63,3096.291,NaN
2019-09-09,BTC,10316.62,10475.54,10077.22,10307.00,14824.373,NaN
2019-09-10,BTC,10307.00,10382.97,9940.87,10102.02,9068.955,0.0002
2019-09-11,BTC,10094.27,10293.11,9884.31,10159.55,10897.922,0.0003
2019-09-12,BTC,10163.06,10450.13,10042.12,10415.13,15609.634,0.0003


### Binance Spot

In [16]:
# pull OHLC from Binance
data_req = DataRequest(source='ccxt',
                       tickers=tickers, 
                       fields=['open', 'high', 'low', 'close', 'volume'], 
                       freq='d')

In [17]:
# df2 = GetData(data_req).get_series()

In [18]:
# df2.to_csv('binance_spot.csv')
df2 = pd.read_csv('../../../../factorlab/notebooks/binance_spot.csv', index_col=['date', 'ticker'], parse_dates=['date'])

In [19]:
df2.head()

open     high      low    close       volume
date       ticker                                                 
2017-08-17 BTC     4261.48  4485.39  4200.74  4285.08   795.150377
           ETH      301.13   312.18   298.00   302.00  7030.710340
2017-08-18 BTC     4285.08  4371.52  3938.77  4108.37  1199.888264
           ETH      302.00   311.79   283.94   293.96  9537.846460
2017-08-19 BTC     4108.37  4184.69  3850.00  4139.98   381.309763

### CryptoCompare - Historical Prices

In [20]:
# pull OHLC from CryptoCompare
data_req = DataRequest(source='cryptocompare',
                       tickers=tickers, 
                       fields=['open', 'high', 'low', 'close', 'volume'], 
                       freq='d')

In [21]:
# df3 = GetData(data_req).get_series()

In [22]:
# df3.to_csv('cc_spot.csv')
df3 = pd.read_csv('../../../../factorlab/notebooks/cc_spot.csv', index_col=['date', 'ticker'], parse_dates=['date'])

### Clean Data

In [61]:
df = stitch_dataframes([df1, df2, df3])
df.funding_rate = df.funding_rate.fillna(0)

In [62]:
df.head()

,,open,high,low,close,volume,funding_rate
date,ticker,,,,,,
2010-07-17,BTC,0.04951,0.04951,0.04951,0.04951,20.00,0.0
2010-07-18,BTC,0.04951,0.08585,0.04951,0.08584,75.01,0.0
2010-07-19,BTC,0.08584,0.09307,0.07723,0.08080,574.00,0.0
2010-07-20,BTC,0.08080,0.08181,0.07426,0.07474,262.00,0.0
2010-07-21,BTC,0.07474,0.07921,0.06634,0.07921,575.00,0.0


In [63]:
# Filter data
clean_df = CleanData(df).filter_outliers(od_method='mad', excl_cols=['volume', 'funding_rate'], thresh_val=10).\
                         repair_outliers(imp_method='fcst').\
                         filter_avg_trading_val(thresh_val=1000000).\
                         filter_missing_vals_gaps().\
                         filter_min_nobs(ts_obs=1000, cs_obs=3).\
                         get(attr='df').dropna(how='all')

In [64]:
clean_df

open     high      low    close        volume  \
date       ticker                                                     
2017-03-21 BTC     1047.51  1125.53  1043.87  1121.29  9.259127e+04   
           ETC       1.867     2.39    1.867    2.378  2.151590e+06   
           ETH       42.51     43.8    41.68    42.67  4.843660e+05   
           LTC       4.121    4.155    4.014     4.09  1.932581e+05   
2017-03-22 BTC     1121.29  1121.88   997.78  1044.72  1.152861e+05   
...                    ...      ...      ...      ...           ...   
2024-08-04 YFI      5341.0   5341.0   5196.0   5198.0  4.281050e+02   
           ZEC       31.76    34.44    31.28    31.55  1.348085e+06   
           ZEN       9.657    9.112    8.285    8.462  2.071124e+06   
           ZIL     0.01422  0.01441  0.01392  0.01396  2.048626e+08   
           ZRX      0.3055   0.3121   0.2983   0.2988  9.810764e+06   

                   funding_rate  
date       ticker                
2017-03-21 BTC         0.000000  
           ETC         0.000000  
           ETH         0.000000  
           LTC         0.000000  
2017-03-22 BTC         0.000000  
...                         ...  
2024-08-04 YFI         0.000169  
           ZEC         0.000191  
           ZEN         0.000194  
           ZIL        -0.000031  
           ZRX         0.000186  

[193002 rows x 6 columns]

In [65]:
# Filter data
clean = CleanData(df)

In [66]:
clean.filter_outliers(od_method='mad', excl_cols=['volume', 'funding_rate'], thresh_val=10).\
                         repair_outliers(imp_method='fcst').\
                         filter_avg_trading_val(thresh_val=1000000).\
                         filter_missing_vals_gaps().\
                         filter_min_nobs(ts_obs=1000, cs_obs=3)

In [79]:
clean.summary.close

,1000SATS,1INCH,AAVE,ACE,ACH,ADA,AEVO,AGIX,AGLD,AI,...,XVG,XVS,YFI,YGG,ZEC,ZEN,ZIL,ZK,ZRO,ZRX
n_obs,237.000000,1319.000000,1395.000000,326.000000,1419.000000,2500.000000,278.000000,536.000000,1040.000000,524.000000,...,3101.000000,1399.000000,1466.000000,1067.000000,2838.000000,2617.000000,2379.000000,49.000000,46.000000,2551.000000
%_NaN_start,95.382817,74.303526,72.822911,93.648938,72.355348,51.295539,94.584064,89.557763,79.738944,89.791545,...,39.586986,72.744983,71.439704,79.212936,44.710695,49.016170,53.652835,99.045393,99.103838,50.301968
%_outliers,0.038964,0.642899,0.370154,0.253263,0.584454,0.876680,0.331190,0.253263,0.526008,0.253263,...,2.824859,0.642899,0.389636,0.467563,0.857199,1.207871,0.974089,0.000000,0.019482,0.993571
%_imputed,0.175336,1.052016,0.584454,0.370154,0.701344,1.246834,0.545490,1.188389,0.779271,0.662381,...,3.779466,0.779271,0.701344,0.740308,1.207871,1.461134,1.422170,0.116891,0.136372,1.363725
%_below_avg_trading_val,0.564972,0.564972,0.603935,1.753361,6.409507,0.564972,2.591077,1.110462,1.636470,6.019871,...,41.671537,4.227547,0.564972,1.402688,4.773037,17.786869,9.526593,0.564972,0.564972,12.195597
%_missing_vals_gaps,0.000000,0.000000,0.000000,0.019482,0.019482,0.000000,0.000000,0.000000,0.000000,0.000000,...,10.325346,13.598286,0.000000,11.903370,16.540035,6.916034,6.623807,0.000000,0.000000,7.305669
n_tickers_below_min_obs,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,...,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000


In [80]:
clean.filter_tickers(['BTC'])

In [85]:
df.drop(['BTC'], level=1, axis=0)

open     high      low    close        volume  \
date       ticker                                                     
2013-09-29 LTC     2.56500  2.59000  2.59000  2.59000  5.000000e+00   
2013-09-30 LTC     2.59000  2.55400  2.55400  2.55400  5.623000e+01   
2013-10-01 LTC     2.55400  2.66600  2.52500  2.66600  1.000000e+00   
2013-10-02 LTC     2.66600  2.33700  2.33700  2.33700  1.500000e+00   
2013-10-03 LTC     2.33700  2.50200  2.48900  2.50200  9.274000e+01   
...                    ...      ...      ...      ...           ...   
2024-08-04 ZEN     8.40600  9.11200  8.28500  8.46200  2.071124e+06   
           ZIL     0.01422  0.01441  0.01392  0.01396  2.048626e+08   
           ZK      0.11451  0.11673  0.10857  0.10874  3.833253e+08   
           ZRO     3.63100  3.64800  3.36200  3.36700  7.049472e+07   
           ZRX     0.30550  0.31210  0.29830  0.29880  9.810764e+06   

                   funding_rate  
date       ticker                
2013-09-29 LTC         0.000000  
2013-09-30 LTC         0.000000  
2013-10-01 LTC         0.000000  
2013-10-02 LTC         0.000000  
2013-10-03 LTC         0.000000  
...                         ...  
2024-08-04 ZEN         0.000194  
           ZIL        -0.000031  
           ZK          0.000200  
           ZRO         0.000184  
           ZRX         0.000186  

[387494 rows x 6 columns]